In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # make detection deterministic

## Reddit Data

In [2]:
reddit_df = pd.read_csv('../data/reddit_crypto_data.csv')
reddit_df

,id,title,text,score,created_utc,author,num_comments,subreddit,permalink,type,platform,parent_id
0,n9cby0,Not every new coin is a shitcoin: How to spot ...,"A few days ago, I made a post titled *""Rugpull...",20342,2021-05-11 02:29:24,hazelvelvet,2467.0,CryptoCurrency,https://reddit.com/r/CryptoCurrency/comments/n...,submission,binance,NaN
1,gxomc6o,NaN,Great post. I have a few questions. \n\n1) How...,11,2021-05-11 10:15:20,hoti0101,NaN,CryptoCurrency,https://reddit.com/r/CryptoCurrency/comments/n...,comment,binance,t3_n9cby0
2,gxnksae,NaN,> Start trading on the BSC (Binance) Smart Cha...,58,2021-05-11 05:05:51,fakesteez,NaN,CryptoCurrency,https://reddit.com/r/CryptoCurrency/comments/n...,comment,binance,t3_n9cby0
3,gxsnol3,NaN,"okay so i bought into 3 coins, around $10 tota...",4,2021-05-12 08:13:54,still_alive11,NaN,CryptoCurrency,https://reddit.com/r/CryptoCurrency/comments/n...,comment,binance,t3_n9cby0
4,gxnazsz,NaN,Isn’t the minimum purchase set to $15 on binance?,7,2021-05-11 03:55:45,BadAssPleb,NaN,CryptoCurrency,https://reddit.com/r/CryptoCurrency/comments/n...,comment,binance,t3_n9cby0
...,...,...,...,...,...,...,...,...,...,...,...,...
58336,guhqwvf,NaN,"Wait, there's an actual Ethereum app? Do I nee...",2,2021-04-14 22:46:45,HallofLogos,NaN,Ethereum,https://reddit.com/r/ethereum/comments/mqr22z/...,comment,coinbase,t3_mqr22z
58337,guhneao,NaN,I see the From and To on [Etherscan.io](https:...,1,2021-04-14 22:19:55,nightwillalwayswin,NaN,Ethereum,https://reddit.com/r/ethereum/comments/mqr22z/...,comment,coinbase,t1_guhm1zp
58338,gui4vqg,NaN,ive been all in on crypto for years and i stil...,20,2021-04-15 00:28:07,None,NaN,Ethereum,https://reddit.com/r/ethereum/comments/mqr22z/...,comment,coinbase,t1_gui4j8p
58339,guiqmeu,NaN,I can't even tell you how happy I am to hear t...,14,2021-04-15 03:06:30,lukejames,NaN,Ethereum,https://reddit.com/r/ethereum/comments/mqr22z/...,comment,coinbase,t1_gui9zsq


In [3]:
# Lets see the unique values in the column 'type'
print(reddit_df['type'].unique())

['submission' 'comment']


In [4]:
# Filter if necessary, on second thought I think its good to keep submissions.
# Submissions are generally more insightful with more information.
# df = df[df['type'] == 'comment']

# Drop rows with missing text of sentiment values
reddit_df = reddit_df.dropna(subset=['text'])

# Combine title and text if both exist
reddit_df['text'] = reddit_df.apply(
    lambda row: (str(row['title']) + " " + str(row['text']))
    if pd.notna(row['title']) and str(row['title']).strip() != str(row['text']).strip()
    else str(row['text']),
    axis=1
)

# Next lets drop unnecessary columns
reddit_df = reddit_df.drop(columns=['title', 'id', 'author', 'num_comments', 'permalink'])

# Drop duplicates
reddit_df = reddit_df.drop_duplicates()
reddit_df

/var/folders/cz/_b8vfkr5561_03c0zfpnzbgc0000gn/T/ipykernel_36507/1336955072.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df['text'] = reddit_df.apply(


,text,score,created_utc,subreddit,type,platform,parent_id
0,Not every new coin is a shitcoin: How to spot ...,20342,2021-05-11 02:29:24,CryptoCurrency,submission,binance,NaN
1,Great post. I have a few questions. \n\n1) How...,11,2021-05-11 10:15:20,CryptoCurrency,comment,binance,t3_n9cby0
2,> Start trading on the BSC (Binance) Smart Cha...,58,2021-05-11 05:05:51,CryptoCurrency,comment,binance,t3_n9cby0
3,"okay so i bought into 3 coins, around $10 tota...",4,2021-05-12 08:13:54,CryptoCurrency,comment,binance,t3_n9cby0
4,Isn’t the minimum purchase set to $15 on binance?,7,2021-05-11 03:55:45,CryptoCurrency,comment,binance,t3_n9cby0
...,...,...,...,...,...,...,...
58336,"Wait, there's an actual Ethereum app? Do I nee...",2,2021-04-14 22:46:45,Ethereum,comment,coinbase,t3_mqr22z
58337,I see the From and To on [Etherscan.io](https:...,1,2021-04-14 22:19:55,Ethereum,comment,coinbase,t1_guhm1zp
58338,ive been all in on crypto for years and i stil...,20,2021-04-15 00:28:07,Ethereum,comment,coinbase,t1_gui4j8p
58339,I can't even tell you how happy I am to hear t...,14,2021-04-15 03:06:30,Ethereum,comment,coinbase,t1_gui9zsq


In [5]:
# Datetime handling
reddit_df['created_utc'] = pd.to_datetime(reddit_df['created_utc']).dt.date
reddit_df

,text,score,created_utc,subreddit,type,platform,parent_id
0,Not every new coin is a shitcoin: How to spot ...,20342,2021-05-11,CryptoCurrency,submission,binance,NaN
1,Great post. I have a few questions. \n\n1) How...,11,2021-05-11,CryptoCurrency,comment,binance,t3_n9cby0
2,> Start trading on the BSC (Binance) Smart Cha...,58,2021-05-11,CryptoCurrency,comment,binance,t3_n9cby0
3,"okay so i bought into 3 coins, around $10 tota...",4,2021-05-12,CryptoCurrency,comment,binance,t3_n9cby0
4,Isn’t the minimum purchase set to $15 on binance?,7,2021-05-11,CryptoCurrency,comment,binance,t3_n9cby0
...,...,...,...,...,...,...,...
58336,"Wait, there's an actual Ethereum app? Do I nee...",2,2021-04-14,Ethereum,comment,coinbase,t3_mqr22z
58337,I see the From and To on [Etherscan.io](https:...,1,2021-04-14,Ethereum,comment,coinbase,t1_guhm1zp
58338,ive been all in on crypto for years and i stil...,20,2021-04-15,Ethereum,comment,coinbase,t1_gui4j8p
58339,I can't even tell you how happy I am to hear t...,14,2021-04-15,Ethereum,comment,coinbase,t1_gui9zsq


In [6]:
CRYPTO_EXCHANGES = {
    'binance': ['binance', 'bnb', 'binance us', 'binance app', 'binance exchange'],
    'coinbase': ['coinbase', 'coinbase pro', 'coinbase wallet', 'cb wallet'],
    'kraken': ['kraken', 'kraken exchange', 'kraken pro'],
    'okx': ['okx', 'okex'],
    'kucoin': ['kucoin', 'kucoin exchange'],
    'crypto.com': ['crypto.com', 'crypto.com app'],
    'bybit': ['bybit', 'bybit app']
}

# Function to detect all mentioned exchanges in a text
def find_exchanges(text: str) -> str:
    text = text.lower()
    matches = []
    for name, keywords in CRYPTO_EXCHANGES.items():
        if any(keyword in text for keyword in keywords):
            matches.append(name)
    return str(matches)

# Apply across all rows and overwrite the 'platform' column
reddit_df['platform'] = reddit_df['text'].apply(find_exchanges)
reddit_df
print(reddit_df['platform'].unique())

["['binance']" "['coinbase']" "['kraken']" "['binance', 'coinbase']"
 "['crypto.com']" "['coinbase', 'kraken']" "['binance', 'crypto.com']"
 "['binance', 'coinbase', 'crypto.com']"
 "['binance', 'coinbase', 'kraken']" "['binance', 'kraken']"
 "['kraken', 'crypto.com']" "['kucoin']" "['binance', 'kucoin']"
 "['kraken', 'kucoin']" "['binance', 'coinbase', 'kraken', 'bybit']"
 "['binance', 'coinbase', 'kucoin']"
 "['coinbase', 'kraken', 'crypto.com']"
 "['binance', 'coinbase', 'kraken', 'kucoin']"
 "['binance', 'kraken', 'crypto.com']" "['coinbase', 'crypto.com']"
 "['binance', 'kraken', 'kucoin']" "['coinbase', 'kraken', 'kucoin']"
 "['binance', 'coinbase', 'kraken', 'crypto.com']"
 "['kucoin', 'crypto.com']" "['coinbase', 'kucoin', 'crypto.com']"
 "['bybit']" "['binance', 'coinbase', 'bybit']"
 "['binance', 'kucoin', 'crypto.com']"
 "['binance', 'kraken', 'kucoin', 'crypto.com']" "['binance', 'bybit']"
 "['binance', 'kucoin', 'bybit']"
 "['binance', 'coinbase', 'kucoin', 'crypto.com']"


## App Store Data

In [7]:
appstore_df = pd.read_csv('../data/app_crypto_data.csv')
appstore_df

,exchange,platform,user,rating,date,text
0,binance,ios,James4all,5,2023-10-04 08:23:28,How can someone set stop lose and take profit ...
1,binance,ios,Zionsb,1,2023-05-24 00:13:10,"I have been using Binance for a little bit, ev..."
2,binance,ios,Username: Courtney,2,2021-04-11 18:25:51,The only thing good about the app is it’s inte...
3,binance,ios,joshdammit,1,2023-06-15 10:57:52,Got an email last week saying binance is shutt...
4,binance,ios,King Banks M,1,2021-05-05 17:13:25,I’ve been trying to buy crypto for a while now...
...,...,...,...,...,...,...
10736,bybit,android,Yamkela Mnqandi,5,2025-02-13 21:17:17,Very helpful app
10737,bybit,android,DAYYABU ALI ADAMU,4,2025-02-13 19:32:09,A wonderful exchange App very well build with ...
10738,bybit,android,Julrex Tumakay,4,2025-02-13 18:52:48,I love this apps
10739,bybit,android,Kolawole Quazim Opeyemi,1,2025-02-13 18:39:41,Just to download the app alone is taking ages ...


In [8]:
# Drop unnecessary columns
appstore_df = appstore_df.drop(columns=['user'])

# Drop rows with missing text or sentiment values
appstore_df = appstore_df.dropna(subset=['text'])

# Drop rows with text length <20
appstore_df = appstore_df[appstore_df['text'].str.split().str.len() >= 20]

In [9]:
# Datetime handling
appstore_df['date'] = pd.to_datetime(appstore_df['date']).dt.date
appstore_df

,exchange,platform,rating,date,text
0,binance,ios,5,2023-10-04,How can someone set stop lose and take profit ...
1,binance,ios,1,2023-05-24,"I have been using Binance for a little bit, ev..."
2,binance,ios,2,2021-04-11,The only thing good about the app is it’s inte...
3,binance,ios,1,2023-06-15,Got an email last week saying binance is shutt...
4,binance,ios,1,2021-05-05,I’ve been trying to buy crypto for a while now...
...,...,...,...,...,...
10727,bybit,android,1,2025-02-14,"Even zero doesn't deserve, after the id succes..."
10728,bybit,android,3,2025-02-14,This is because I've been trying to sign in a ...
10729,bybit,android,2,2025-02-14,The contact seller messenger is too hard to up...
10732,bybit,android,5,2025-02-14,starting my crypto journey on bybit. Everythin...


## TrustPilot Reviews Data

In [10]:
tpilot_df = pd.read_csv('../data/trustpilot_crypto_data.csv')
tpilot_df

,Exchange,Username,Date,Review,Rating,Sentiment
0,binance,Akm LvL,"March 20, 2025",your account with your balance will disappear ...,1,neutral
1,binance,Rubeek,"March 20, 2025",It should improve for retailers instead of jus...,3,neutral
2,binance,Sophia,"March 21, 2025",I have been using this platform for few years ...,3,positive
3,binance,jaco antony,"March 01, 2025",man i had headache to withdrawl,3,neutral
4,binance,John Haines,"January 01, 2025","Getting money in is easy, but out is ridiculou...",1,positive
...,...,...,...,...,...,...
12008,bybit,Jckscott,"May 21, 2022",I loved my time trading on this website everyt...,5,positive
12009,bybit,Luca,"October 13, 2022",I've Lost more than 900$ for a long order clos...,1,positive
12010,bybit,GAME CHANGER,"October 11, 2022",Scam exchange has scammed me 114$ on p2p and n...,1,neutral
12011,bybit,Mircea,"August 24, 2022",My money were lost via P2P transaction.The sel...,1,negative


In [11]:
# Drop unnecessary columns
tpilot_df = tpilot_df.drop(columns=['Username'])

# Drop rows with missing review or sentiment values
tpilot_df = tpilot_df.dropna(subset=['Review', 'Sentiment'])

# Drop rows where the review has <20 words
tpilot_df = tpilot_df[tpilot_df['Review'].str.split().str.len() >= 20]
tpilot_df

,Exchange,Date,Review,Rating,Sentiment
2,binance,"March 21, 2025",I have been using this platform for few years ...,3,positive
4,binance,"January 01, 2025","Getting money in is easy, but out is ridiculou...",1,positive
5,binance,"March 20, 2025",Account disappeared. Circular nightmare trying...,1,negative
6,binance,"March 19, 2025",Do not recommend to anybody!They got celphone ...,1,negative
8,binance,"March 24, 2024",There are those people they can't make single ...,5,negative
...,...,...,...,...,...
12007,bybit,"October 21, 2022","hello, my bybit account is restricted or banne...",1,positive
12008,bybit,"May 21, 2022",I loved my time trading on this website everyt...,5,positive
12009,bybit,"October 13, 2022",I've Lost more than 900$ for a long order clos...,1,positive
12010,bybit,"October 11, 2022",Scam exchange has scammed me 114$ on p2p and n...,1,neutral


In [12]:
# Datetime handling
tpilot_df['Date'] = pd.to_datetime(tpilot_df['Date']).dt.date
tpilot_df

,Exchange,Date,Review,Rating,Sentiment
2,binance,2025-03-21,I have been using this platform for few years ...,3,positive
4,binance,2025-01-01,"Getting money in is easy, but out is ridiculou...",1,positive
5,binance,2025-03-20,Account disappeared. Circular nightmare trying...,1,negative
6,binance,2025-03-19,Do not recommend to anybody!They got celphone ...,1,negative
8,binance,2024-03-24,There are those people they can't make single ...,5,negative
...,...,...,...,...,...
12007,bybit,2022-10-21,"hello, my bybit account is restricted or banne...",1,positive
12008,bybit,2022-05-21,I loved my time trading on this website everyt...,5,positive
12009,bybit,2022-10-13,I've Lost more than 900$ for a long order clos...,1,positive
12010,bybit,2022-10-11,Scam exchange has scammed me 114$ on p2p and n...,1,neutral


In [13]:
# It has been verified that all is in English, when crawling I forgot to only accept data if its in English. But all data is in English.
'''
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False  # in case of empty or corrupt input

# Apply to the DataFrame
tpilot_df = tpilot_df[tpilot_df['Review'].apply(is_english)]
tpilot_df
'''

"\ndef is_english(text):\n    try:\n        return detect(text) == 'en'\n    except:\n        return False  # in case of empty or corrupt input\n\n# Apply to the DataFrame\ntpilot_df = tpilot_df[tpilot_df['Review'].apply(is_english)]\ntpilot_df\n"

## Aggregation

In [14]:
reddit_df['subreddit'] = 'r/' + reddit_df['subreddit'].astype(str)
reddit_df = reddit_df.rename(columns={
    'created_utc': 'Date',
    'subreddit': 'Source', # subreddit name
    'platform': 'Exchange',  # exchange or 'general'
    'text': 'Text',
    'score': 'Reddit Score',
    'sentiment': 'Sentiment',
    'type': 'Type',
    'parent_id': 'Parent ID'
})
reddit_df['Platform'] = 'web'
reddit_df['Rating'] = None
reddit_df

,Text,Reddit Score,Date,Source,Type,Exchange,Parent ID,Platform,Rating
0,Not every new coin is a shitcoin: How to spot ...,20342,2021-05-11,r/CryptoCurrency,submission,['binance'],NaN,web,None
1,Great post. I have a few questions. \n\n1) How...,11,2021-05-11,r/CryptoCurrency,comment,['binance'],t3_n9cby0,web,None
2,> Start trading on the BSC (Binance) Smart Cha...,58,2021-05-11,r/CryptoCurrency,comment,['binance'],t3_n9cby0,web,None
3,"okay so i bought into 3 coins, around $10 tota...",4,2021-05-12,r/CryptoCurrency,comment,['binance'],t3_n9cby0,web,None
4,Isn’t the minimum purchase set to $15 on binance?,7,2021-05-11,r/CryptoCurrency,comment,['binance'],t3_n9cby0,web,None
...,...,...,...,...,...,...,...,...,...
58336,"Wait, there's an actual Ethereum app? Do I nee...",2,2021-04-14,r/Ethereum,comment,['coinbase'],t3_mqr22z,web,None
58337,I see the From and To on [Etherscan.io](https:...,1,2021-04-14,r/Ethereum,comment,['coinbase'],t1_guhm1zp,web,None
58338,ive been all in on crypto for years and i stil...,20,2021-04-15,r/Ethereum,comment,['coinbase'],t1_gui4j8p,web,None
58339,I can't even tell you how happy I am to hear t...,14,2021-04-15,r/Ethereum,comment,['coinbase'],t1_gui9zsq,web,None


In [15]:
appstore_df = appstore_df.rename(columns={
    'exchange': 'Exchange',
    'platform': 'Platform',  # already okay
    'rating': 'Rating',
    'date': 'Date',
    'text': 'Text',
    'sentiment': 'Sentiment'
})
appstore_df['Source'] = appstore_df['Platform'].apply(
    lambda x: 'app store' if x == 'ios' else 'play store'
)
appstore_df['Type'] = appstore_df['Platform'].apply(
    lambda x: 'app store review' if x == 'ios' else 'play store review'
)
appstore_df['Reddit Score'] = None
appstore_df['Parent ID'] = None
appstore_df

,Exchange,Platform,Rating,Date,Text,Source,Type,Reddit Score,Parent ID
0,binance,ios,5,2023-10-04,How can someone set stop lose and take profit ...,app store,app store review,None,None
1,binance,ios,1,2023-05-24,"I have been using Binance for a little bit, ev...",app store,app store review,None,None
2,binance,ios,2,2021-04-11,The only thing good about the app is it’s inte...,app store,app store review,None,None
3,binance,ios,1,2023-06-15,Got an email last week saying binance is shutt...,app store,app store review,None,None
4,binance,ios,1,2021-05-05,I’ve been trying to buy crypto for a while now...,app store,app store review,None,None
...,...,...,...,...,...,...,...,...,...
10727,bybit,android,1,2025-02-14,"Even zero doesn't deserve, after the id succes...",play store,play store review,None,None
10728,bybit,android,3,2025-02-14,This is because I've been trying to sign in a ...,play store,play store review,None,None
10729,bybit,android,2,2025-02-14,The contact seller messenger is too hard to up...,play store,play store review,None,None
10732,bybit,android,5,2025-02-14,starting my crypto journey on bybit. Everythin...,play store,play store review,None,None


In [16]:
tpilot_df = tpilot_df.rename(columns={
    'Review': 'Text',
})
tpilot_df['Platform'] = 'web'
tpilot_df['Source'] = 'trustpilot'
tpilot_df['Reddit Score'] = None
tpilot_df['Parent ID'] = None
tpilot_df['Type'] = 'trustpilot review'
tpilot_df

,Exchange,Date,Text,Rating,Sentiment,Platform,Source,Reddit Score,Parent ID,Type
2,binance,2025-03-21,I have been using this platform for few years ...,3,positive,web,trustpilot,None,None,trustpilot review
4,binance,2025-01-01,"Getting money in is easy, but out is ridiculou...",1,positive,web,trustpilot,None,None,trustpilot review
5,binance,2025-03-20,Account disappeared. Circular nightmare trying...,1,negative,web,trustpilot,None,None,trustpilot review
6,binance,2025-03-19,Do not recommend to anybody!They got celphone ...,1,negative,web,trustpilot,None,None,trustpilot review
8,binance,2024-03-24,There are those people they can't make single ...,5,negative,web,trustpilot,None,None,trustpilot review
...,...,...,...,...,...,...,...,...,...,...
12007,bybit,2022-10-21,"hello, my bybit account is restricted or banne...",1,positive,web,trustpilot,None,None,trustpilot review
12008,bybit,2022-05-21,I loved my time trading on this website everyt...,5,positive,web,trustpilot,None,None,trustpilot review
12009,bybit,2022-10-13,I've Lost more than 900$ for a long order clos...,1,positive,web,trustpilot,None,None,trustpilot review
12010,bybit,2022-10-11,Scam exchange has scammed me 114$ on p2p and n...,1,neutral,web,trustpilot,None,None,trustpilot review


In [17]:
combined_df = pd.concat([reddit_df, appstore_df, tpilot_df], ignore_index=True)
combined_df = combined_df.drop_duplicates()
combined_df

,Text,Reddit Score,Date,Source,Type,Exchange,Parent ID,Platform,Rating,Sentiment
0,Not every new coin is a shitcoin: How to spot ...,20342,2021-05-11,r/CryptoCurrency,submission,['binance'],NaN,web,None,NaN
1,Great post. I have a few questions. \n\n1) How...,11,2021-05-11,r/CryptoCurrency,comment,['binance'],t3_n9cby0,web,None,NaN
2,> Start trading on the BSC (Binance) Smart Cha...,58,2021-05-11,r/CryptoCurrency,comment,['binance'],t3_n9cby0,web,None,NaN
3,"okay so i bought into 3 coins, around $10 tota...",4,2021-05-12,r/CryptoCurrency,comment,['binance'],t3_n9cby0,web,None,NaN
4,Isn’t the minimum purchase set to $15 on binance?,7,2021-05-11,r/CryptoCurrency,comment,['binance'],t3_n9cby0,web,None,NaN
...,...,...,...,...,...,...,...,...,...,...
71965,"hello, my bybit account is restricted or banne...",None,2022-10-21,trustpilot,trustpilot review,bybit,None,web,1,positive
71966,I loved my time trading on this website everyt...,None,2022-05-21,trustpilot,trustpilot review,bybit,None,web,5,positive
71967,I've Lost more than 900$ for a long order clos...,None,2022-10-13,trustpilot,trustpilot review,bybit,None,web,1,positive
71968,Scam exchange has scammed me 114$ on p2p and n...,None,2022-10-11,trustpilot,trustpilot review,bybit,None,web,1,neutral


In [18]:
print(combined_df['Exchange'].unique())

["['binance']" "['coinbase']" "['kraken']" "['binance', 'coinbase']"
 "['crypto.com']" "['coinbase', 'kraken']" "['binance', 'crypto.com']"
 "['binance', 'coinbase', 'crypto.com']"
 "['binance', 'coinbase', 'kraken']" "['binance', 'kraken']"
 "['kraken', 'crypto.com']" "['kucoin']" "['binance', 'kucoin']"
 "['kraken', 'kucoin']" "['binance', 'coinbase', 'kraken', 'bybit']"
 "['binance', 'coinbase', 'kucoin']"
 "['coinbase', 'kraken', 'crypto.com']"
 "['binance', 'coinbase', 'kraken', 'kucoin']"
 "['binance', 'kraken', 'crypto.com']" "['coinbase', 'crypto.com']"
 "['binance', 'kraken', 'kucoin']" "['coinbase', 'kraken', 'kucoin']"
 "['binance', 'coinbase', 'kraken', 'crypto.com']"
 "['kucoin', 'crypto.com']" "['coinbase', 'kucoin', 'crypto.com']"
 "['bybit']" "['binance', 'coinbase', 'bybit']"
 "['binance', 'kucoin', 'crypto.com']"
 "['binance', 'kraken', 'kucoin', 'crypto.com']" "['binance', 'bybit']"
 "['binance', 'kucoin', 'bybit']"
 "['binance', 'coinbase', 'kucoin', 'crypto.com']"


In [19]:
combined_df.to_csv('../data/crypto_exchange_data_raw.csv', index=False)